In [0]:
# Gold external location
gold_path = "abfss://datalake@etldatalakeabhi.dfs.core.windows.net/gold/"

# Check Silver table
silver_table = "ws_databricks_etl.silver.orders"


In [0]:
# Gold external location
gold_path = "abfss://datalake@etldatalakeabhi.dfs.core.windows.net/gold/"

# Check Silver table
silver_table = "ws_databricks_etl.silver.orders"


In [0]:
# Read cleaned Silver orders
silver_orders = spark.read.table(silver_table)
display(silver_orders)


In [0]:
from pyspark.sql.functions import sum as _sum, count as _count, col, when

# Total sales and orders per customer
gold_customer_sales = silver_orders.groupBy("customer_id").agg(
    _sum("amount").alias("total_sales"),
    _count("order_id").alias("total_orders"),
    _sum(when(col("status") == "cancelled", 1).otherwise(0)).alias("cancelled_orders")
)

# Optional: add processing timestamp
from pyspark.sql.functions import current_timestamp
gold_customer_sales = gold_customer_sales.withColumn("processed_timestamp", current_timestamp())

display(gold_customer_sales)


In [0]:
gold_customer_sales.write.format("delta").mode("overwrite").save(gold_path + "customer_sales/")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS ws_databricks_etl.gold.customer_sales
USING DELTA
LOCATION '{gold_path}customer_sales/'
""")

print("Gold table registered successfully in Unity Catalog")


In [0]:
%sql
select * from ws_databricks_etl.gold.customer_sales